In [1]:
!git clone https://github.com/ultralytics/yolov5.git



fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
%pip install torch 


In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: C:\Users\abhay\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from pathlib import Path

model_path = Path(r"C:\Users\abhay\OneDrive\Desktop\New folder\iitm\Python-Abhay\minor_project\best.pt")
image_path = Path(r"C:\Users\abhay\OneDrive\Desktop\New folder\iitm\Python-Abhay\minor_project\car.png")

print("Model path:", model_path.resolve())
print("Exists?  ", model_path.exists())
print("Image path:", image_path.resolve())
print("Exists?   ", image_path.exists())

Model path: C:\Users\abhay\OneDrive\Desktop\New folder\iitm\Python-Abhay\minor_project\best.pt
Exists?   True
Image path: C:\Users\abhay\OneDrive\Desktop\New folder\iitm\Python-Abhay\minor_project\car.png
Exists?    True


In [4]:
import torch
import pathlib
from pathlib import Path
from PIL import Image
import numpy as np
import easyocr

# --- Fix for Windows 'PosixPath' issue ---
pathlib.PosixPath = pathlib.WindowsPath

# --- paths ---
model_path = Path(r"C:\Users\abhay\OneDrive\Desktop\New folder\iitm\Python-Abhay\minor_project\best.pt")
image_path = Path(r"C:\Users\abhay\OneDrive\Desktop\New folder\iitm\Python-Abhay\minor_project\car.png")
out_dir = Path("runs/plates")
out_dir.mkdir(parents=True, exist_ok=True)

# --- load YOLOv5 model ---
model = torch.hub.load('ultralytics/yolov5', 'custom',
                       path=str(model_path), trust_repo=True, source='github')

# --- detection parameters ---
model.conf = 0.15
model.iou = 0.45
model.max_det = 100

# --- run inference ---
results = model(str(image_path), size=1280)
results.save()

# --- extract detection dataframe ---
df = results.pandas().xyxy[0]
print(df)

# --- determine which classes are plates ---
plate_names = {
    n for n in set(df['name'].astype(str).str.lower())
    if 'plate' in n or 'licence' in n or 'license' in n or 'number' in n
}
if not plate_names and len(model.names) == 1:
    plate_names = {list(model.names.values())[0].lower()}

print("Plate class names detected:", plate_names)

# --- crop and save plates ---
img = Image.open(image_path).convert("RGB")
saved = []
for i, row in df.iterrows():
    if str(row['name']).lower() in plate_names:
        x1, y1, x2, y2 = map(int, [row.xmin, row.ymin, row.xmax, row.ymax])
        crop = img.crop((x1, y1, x2, y2))
        cp = out_dir / f"plate_{i}_{row['confidence']:.2f}.png"
        crop.save(cp)
        saved.append(cp)

# --- OCR using EasyOCR ---
if saved:
    print("\n✅ Cropped plate images saved:")
    reader = easyocr.Reader(['en'], gpu=False)  # set gpu=True if you have CUDA
    for p in saved:
        print(f"\n🔹 Reading text from: {p.name}")
        
        # --- FIX: read image as numpy array for EasyOCR ---
        img_np = np.array(Image.open(p).convert("RGB"))
        result = reader.readtext(img_np)
        
        if not result:
            print("   ⚠️ No text detected.")
            continue

        for (bbox, text, conf) in result:
            print(f"   Detected: '{text}' (confidence={conf:.2f})")
else:
    print("\n⚠️ No plates cropped. Try:")
    print("  • Lowering model.conf to 0.05")
    print("  • Increasing inference size to 1280 or 1536")
    print("  • Check your model’s class names")


Using cache found in C:\Users\abhay/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-10-21 Python-3.10.11 torch-2.9.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
C:\Users\abhay/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Saved 1 image to runs\detect\exp13
Using CPU. Note: This module is much faster with a GPU.


         xmin        ymin      xmax        ymax  confidence  class   name
0  164.847092  200.300583  238.8703  219.460617    0.857743      0  plate
Plate class names detected: {'plate'}

✅ Cropped plate images saved:

🔹 Reading text from: plate_0_0.86.png
   Detected: 'HR 26 BC 5514' (confidence=0.93)


In [ ]:
import torch
import cv2
import easyocr
from PIL import Image
import numpy as np

# --- Load YOLOv5 custom model ---
model_path = r"C:\Users\abhay\OneDrive\Desktop\New folder\iitm\Python-Abhay\minor_project\best.pt"
model = torch.hub.load('ultralytics/yolov5', 'custom',
                       path=model_path, source='github', trust_repo=True)

# Lower confidence threshold if needed
model.conf = 0.15
model.iou = 0.45

# --- Initialize EasyOCR ---
reader = easyocr.Reader(['en'], gpu=True)

# --- Open webcam ---
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Cannot access webcam.")
    exit()

print("🎥 Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Failed to grab frame.")
        break

    # --- Run YOLOv5 inference ---
    results = model(frame)

    # Convert results to pandas DataFrame
    df = results.pandas().xyxy[0]

    # Assume plate class name
    plate_names = {n for n in set(df['name'].astype(str).str.lower())
                   if 'plate' in n or 'number' in n or 'license' in n or 'licence' in n}
    if not plate_names and len(model.names) == 1:
        plate_names = {list(model.names.values())[0].lower()}

    # --- Process detections ---
    for i, row in df.iterrows():
        name = str(row['name']).lower()
        if name in plate_names:
            x1, y1, x2, y2 = map(int, [row.xmin, row.ymin, row.xmax, row.ymax])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Crop detected plate
            crop = frame[y1:y2, x1:x2]

            # --- Run OCR on cropped plate ---
            if crop.size > 0:
                result = reader.readtext(crop)
                if result:
                    # Take the text with highest confidence
                    best = max(result, key=lambda r: r[2])
                    text = best[1]
                    conf = best[2]

                    # Display text on frame
                    cv2.putText(frame, f"{text} ({conf:.2f})",
                                (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                                (255, 0, 0), 2, cv2.LINE_AA)
                    print(f"🔹 Detected Plate: {text}  (Conf: {conf:.2f})")

    # --- Display frame ---
    cv2.imshow("YOLOv5 Plate Detection + OCR", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



Using cache found in C:\Users\abhay/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-10-21 Python-3.10.11 torch-2.2.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


🎥 Press 'q' to quit.


In [ ]:
import torch
import pathlib
from pathlib import Path
from PIL import Image
import numpy as np
import easyocr
import os
import time
 
#arduino comms setup
import serial.tools.list_ports
 
ports = serial.tools.list_ports.comports()
serialInst = serial.Serial()
portsList = []
 
def match(text,reg):
    6
 
 
for one in ports:
    portsList.append(str(one))
    print(str(one))
 
com = input("Select Com Port for Arduino #: ")
 
for i in range(len(portsList)):
    if portsList[i].startswith("COM" + str(com)):
        use = "COM" + str(com)
        print(use)
 
serialInst.baudrate = 115200
serialInst.port = use
serialInst.open()
reg_list=["tn09cb5990","br19g8869","mn03t2532","access333","hr26bc5514"]














# --- Fix for Windows 'PosixPath' issue ---
pathlib.PosixPath = pathlib.WindowsPath

# --- paths ---
model_path = Path(r"C:\Users\abhay\OneDrive\Desktop\New folder\iitm\Python-Abhay\minor_project\best.pt")
image_path = Path(r"C:\Users\abhay\OneDrive\Desktop\New folder\iitm\Python-Abhay\minor_project\car.png")
out_dir = Path("runs/plates")
out_dir.mkdir(parents=True, exist_ok=True)

# --- load YOLOv5 model ---
model = torch.hub.load('ultralytics/yolov5', 'custom',
                       path=str(model_path), trust_repo=True, source='github')

# --- detection parameters ---
model.conf = 0.15
model.iou = 0.45
model.max_det = 100

# --- run inference ---
results = model(str(image_path), size=1280)
results.save()

# --- extract detection dataframe ---
df = results.pandas().xyxy[0]
print(df)

# --- determine which classes are plates ---
plate_names = {
    n for n in set(df['name'].astype(str).str.lower())
    if 'plate' in n or 'licence' in n or 'license' in n or 'number' in n
}
if not plate_names and len(model.names) == 1:
    plate_names = {list(model.names.values())[0].lower()}

print("Plate class names detected:", plate_names)

# --- crop and save plates ---
img = Image.open(image_path).convert("RGB")
saved = []
for i, row in df.iterrows():
    if str(row['name']).lower() in plate_names:
        x1, y1, x2, y2 = map(int, [row.xmin, row.ymin, row.xmax, row.ymax])
        crop = img.crop((x1, y1, x2, y2))
        cp = out_dir / f"plate_{i}_{row['confidence']:.2f}.png"
        crop.save(cp)
        saved.append(cp)

# --- OCR using EasyOCR ---
if saved:
    print("\n✅ Cropped plate images saved:")
    reader = easyocr.Reader(['en'], gpu=False)  # set gpu=True if you have CUDA
    for p in saved:
        print(f"\n🔹 Reading text from: {p.name}")
        
        # --- FIX: read image as numpy array for EasyOCR ---
        img_np = np.array(Image.open(p).convert("RGB"))
        result = reader.readtext(img_np)
        
        if not result:
            print("   ⚠️ No text detected.")
            continue

        for (bbox, text, conf) in result:
            print(f"   Detected: '{text}' (confidence={conf:.2f})")
else:
    print("\n No plates cropped. Try:")
    print("  • Lowering model.conf to 0.05")
    print("  • Increasing inference size to 1280 or 1536")
    print("  • Check your model’s class names")


text2 = text.replace(" ","")
text2 = text2.lower()   
print(text2)                                              #arduino comms
if text2 in reg_list:
    command="ON"

else:
    command="OFF"
serialInst.write(command.encode('utf-8'))
print(command)



COM5 - Silicon Labs CP210x USB to UART Bridge (COM5)
